In [2]:
import os
import stata_setup
## 设置pystata目录
stata_setup.config(os.getenv("STATA_SYSDIR"), 'mp')
from pystata import stata

下面以数据集 cardkrueger1994.dta 为例。该数据集来自 Card and Krueger (1994) 的自然实验。被解释变量为 fte (full time employment 人数)，实验组虚拟变量为 treated (treated = 1, 如果快餐店在新泽西州; treated = 0, 如果快餐店在宾夕法尼亚州)，实验期虚拟变量为 \( t \) ( t = 1, 如果时间为 1992 年 11 月; t = 0, 如果时间为 1992 年 2 月)。其他解释变量包括以下快餐品牌虚拟变量: bk (Burger King = 1), kfc (Kentucky Fried Chicken = 1), roys (Roy Rogers = 1), wendys (Wendy's = 1)。首先，手工计算双重差分法。


In [4]:
stata.run('use cardkrueger1994.dta, replace')

(Dataset from Card&Krueger (1994))


In [5]:
%stata gen gd=t*treated

In [6]:
%stata reg fte gd treated t,r


Linear regression                               Number of obs     =        801
                                                F(3, 797)         =       1.43
                                                Prob > F          =     0.2330
                                                R-squared         =     0.0080
                                                Root MSE          =      9.003

------------------------------------------------------------------------------
             |               Robust
         fte | Coefficient  std. err.      t    P>|t|     [95% conf. interval]
-------------+----------------------------------------------------------------
          gd |   2.913982   1.736818     1.68   0.094    -.4952963    6.323261
     treated |  -2.883534   1.403338    -2.05   0.040    -5.638209   -.1288592
           t |   -2.40651   1.594091    -1.51   0.132    -5.535623    .7226031
       _cons |   19.94872   1.317281    15.14   0.000     17.36297    22.53447
--------------

从上表可知，处理效应（即互动项 gd）的系数为 2.914，但仅在 10\% 水平上显著（ p 值为 0.094）。其次，使用命令 diff 来估计。

In [7]:
%stata diff fte, t(treated) p(t) robust


DIFFERENCE-IN-DIFFERENCES ESTIMATION RESULTS
--------------------------------------------
Number of observations in the DIFF-IN-DIFF: 801
            Before         After    
   Control: 78             77          155
   Treated: 326            320         646
            404            397
--------------------------------------------------------
 Outcome var.   | fte     | S. Err. |   |t|   |  P>|t|
----------------+---------+---------+---------+---------
Before          |         |         |         | 
   Control      | 19.949  |         |         | 
   Treated      | 17.065  |         |         | 
   Diff (T-C)   | -2.884  | 1.403   | -2.05   | 0.040**
After           |         |         |         | 
   Control      | 17.542  |         |         | 
   Treated      | 17.573  |         |         | 
   Diff (T-C)   | 0.030   | 1.023   | 0.03    | 0.976
                |         |         |         | 
Diff-in-Diff    | 2.914   | 1.737   | 1.68    | 0.094*
------------------------------

上表第一列显示，处理效应的系数估计值为 2.914，而 p 值为 0.094，这与手工回归的结果完全相同。上表将“t = 0”的时期称为“Baseline”（基期），而将“t = 1”的时期称为“Follow-up”（跟踪期）。再次，引入快餐品牌虚拟变量，再进行手工回归（以 wendys 作为参照虚拟变量）。

In [8]:
%stata reg fte gd treated t bk kfc roys, r


Linear regression                               Number of obs     =        801
                                                F(6, 794)         =      57.30
                                                Prob > F          =     0.0000
                                                R-squared         =     0.1878
                                                Root MSE          =     8.1617

------------------------------------------------------------------------------
             |               Robust
         fte | Coefficient  std. err.      t    P>|t|     [95% conf. interval]
-------------+----------------------------------------------------------------
          gd |    2.93502   1.543422     1.90   0.058    -.0946504     5.96469
     treated |  -2.323906   1.253701    -1.85   0.064    -4.784867    .1370549
           t |  -2.402678   1.410265    -1.70   0.089    -5.170966    .3656108
          bk |   .9168795   .9382545     0.98   0.329    -.9248729    2.758632
         kfc |

从上表可知，在引入其他解释变量（即快餐品牌虚拟变量）后，处理效应的系数估计值增大为2.935，并接近5%的显著性水平（p值为0.058）。下面再次使用命令diff。

In [9]:
%stata diff fte, t(treated) p(t) cov(bk kfc roys) robust

DIFFERENCE-IN-DIFFERENCES WITH COVARIATES
-----------------------------------------

DIFFERENCE-IN-DIFFERENCES ESTIMATION RESULTS
--------------------------------------------
Number of observations in the DIFF-IN-DIFF: 801
            Before         After    
   Control: 78             77          155
   Treated: 326            320         646
            404            397
--------------------------------------------------------
 Outcome var.   | fte     | S. Err. |   |t|   |  P>|t|
----------------+---------+---------+---------+---------
Before          |         |         |         | 
   Control      | 21.161  |         |         | 
   Treated      | 18.837  |         |         | 
   Diff (T-C)   | -2.324  | 1.254   | -1.85   | 0.064*
After           |         |         |         | 
   Control      | 18.758  |         |         | 
   Treated      | 19.369  |         |         | 
   Diff (T-C)   | 0.611   | 0.900   | 0.68    | 0.497
                |         |         |         | 
Di

上表第一列显示的处理效应估计结果与手工回归完全一样。最后，使用命令 diff 检验各变量在实验组与控制组的均值是否相等。

In [14]:
stata.run(
    """
    set linesize 100
    diff fte, t(treated) p(t) cov(bk kfc roys wendys) test
    """
)


. 
.     set linesize 100

.     diff fte, t(treated) p(t) cov(bk kfc roys wendys) test
TWO-SAMPLE T TEST
-----------------

Number of observations (baseline): 404
            Before         After    
   Control: 78             -           78
   Treated: 326            -           326
            404            -

t-test at period = 0:
----------------------------------------------------------------------------------------------
 Variable(s)         |   Mean Control   | Mean Treated |    Diff.   |   |t|   |  Pr(|T|>|t|)
---------------------+------------------+--------------+------------+---------+---------------
fte                  | 19.949           | 17.065       | -2.884     |  2.44   | 0.0150**
bk                   | 0.436            | 0.408        | -0.028     |  0.45   | 0.6538
kfc                  | 0.154            | 0.209        | 0.055      |  1.09   | 0.2769
roys                 | 0.218            | 0.252        | 0.034      |  0.62   | 0.5368
wendys               | 0.192

从上表可以看出，在基期（即实验前的时期），结果变量 fte（全职员工人数）的均值在两组之间存在显著差异。然而，各品牌虚拟变量的均值在两组之间没有显著差异。

在许多情况下，并没有随机实验或自然实验的数据，而只有观测数据（observational data）。例如，政府提供的就业培训项目，完全由个人决定是否参与。此时，由于实际处理水平$x_i$存在自我选择（self selection），并非随机分组，故可能导致不一致的估计。因此，对于观测数据，需使用特别的计量方法来估计处理效应，详见第28章